## Obtaining Zipcodes ##

**If we want zipcodes..**

In [ ]:
pip install geopy

In [12]:
#import libraries
import numpy as np
import json
import pandas as pd
#from uszipcode import SearchEngine, SimpleZipcode, Zipcode
from uszipcode import SearchEngine
import sqlite3
#from geopy.geocoders import Nominatim

In [2]:
#load up clean dataset
with open('aqs_PM2.5_data_clean.json') as f:
    data=json.load(f)

In [3]:
#turn json file into dataframe
aqs_df = pd.DataFrame(data)

In [5]:
len(aqs_df['latitude'].unique())

139

In [6]:
len(aqs_df['longitude'].unique())

139

### Code from Joel

In [14]:
search =  SearchEngine(db_file_dir="/tmp/db")

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


In [15]:
conn = sqlite3.connect("/tmp/db/simple_db.sqlite")

In [16]:
# pandas.read_sql_query(sql, conn,)

#I don't know what our sql would be? -Jacy
pdf = pd.read_sql_query("select  zipcode, lat, lng, radius_in_miles, 
                        bounds_west, bounds_east, bounds_north, bounds_south from 
                        simple_zipcode",conn)

SyntaxError: EOL while scanning string literal (<ipython-input-16-f85c1b03848d>, line 1)

In [17]:
brd_pdf = sc.broadcast(pdf)
#@udf('string')

NameError: name 'sc' is not defined

In [18]:
def get_zip_b(lat, lng):
    pdf = brd_pdf.value
    try:
        out = pdf[(pdf['bounds_north']>=lat) & 
                  (pdf['bounds_south']<=lat) & 
                  (pdf['bounds_west']<=lng) &  
                  (pdf['bounds_east']>=lng) ]
        dist = [None]*len(out)
        for i in range(len(out)):
            dist[i] = (out['lat'].iloc[i]-lat)**2 + (out['lng'].iloc[i]-lng)**2
        zip = out['zipcode'].iloc[dist.index(min(dist))]
    except:
        zip = 'bad'
    return zip

In [19]:
output_df = df.withColumn('zip', get_zip_b(col("latitude"),col("longitude"))).cache()

NameError: name 'df' is not defined

### Code from Edgar

In [4]:
#Returns a list of zipcodes with a radious
# radius is default zero
# if the we get nothing back it will have 'NONE' so we can find those rows later to process again
def get_zips(lat,lng,r=30):
    search = SearchEngine()
    results = search.by_coordinates(lat=lat,lng=lng, radius=r)
    return [r.zipcode for r in results] if len(results)!=0 else 'NONE'

def get_zips_city(city):
    search = SearchEngine()
    results = search.by_city(city=city)
    return [r.zipcode for r in results] if len(results)!=0 else 'NONE'

In [11]:
#make list of coordinates
values = list(zip(aqs_df['latitude'], aqs_df['longitude']))

In [ ]:
zip_codes = [','.join(get_zips(lat,lng,1)) for lat,lng in values]
#zip_codes = [print(lat) for lat,lng in values[:10]]
#get_zips(35, -120, 1)

In [42]:
zip_codes = []

In [43]:
for lat,lng in values:
    temp = get_zips(lat,lng,1)
    if len(temp)==0:
        zip_codes.append('None')
    else:
        zip_codes.append(','.join(get_zips(lat,lng,1)))

In [44]:
zip_codes

['N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,O,N,E',
 'N,

### Add zipcode list to aqs_df and turn into json

In [45]:
aqs_df['zipcodes'] = zip_codes

In [48]:
aqs_df.tail()

,date_local,county,city,latitude,longitude,aqi,units_of_measure,first_max_value,first_max_hour,zipcodes
1076963,1609545600000,San Luis Obispo,Arroyo Grande,35.04673,-120.58777,39.0,Micrograms/cubic meter (LC),9.4,0,"N,O,N,E"
1076964,1609459200000,San Luis Obispo,Arroyo Grande,35.04673,-120.58777,40.0,Micrograms/cubic meter (LC),9.6,0,"N,O,N,E"
1076965,1609459200000,San Luis Obispo,Arroyo Grande,35.04673,-120.58777,40.0,Micrograms/cubic meter (LC),9.6,0,"N,O,N,E"
1076966,1609459200000,San Luis Obispo,Arroyo Grande,35.04673,-120.58777,40.0,Micrograms/cubic meter (LC),9.6,0,"N,O,N,E"
1076967,1609459200000,San Luis Obispo,Arroyo Grande,35.04673,-120.58777,40.0,Micrograms/cubic meter (LC),9.6,0,"N,O,N,E"


In [47]:
aqs_df['zipcodes'].unique()

array(['N,O,N,E', '95818,95811,95814', '95817', '95661', '91106,91108',
       '90807', '90755', '93726', '93612', '93309', '91360', '91335',
       '90012', '91910', '91506,91502', '93614', '94801', '95062',
       '95205', '93103', '94158,94107', '94603', '94612', '94590',
       '93301', '95018', '95122,95116', '94606', '91762', '94710,94702',
       '94588'], dtype=object)

In [51]:
aqs_df['zipcodes'] = aqs_df['zipcodes'].replace({'N,O,N,E': None})

In [53]:
aqs_df.to_json('aqs_zipcode.json')